In [2]:
from pdfextractor import extractor
import pdfextractor
import pickle
import jieba
import pandas as pd
import re
import sys
import pickle
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout
import random
import requests
import operator
import numpy as np

In [3]:
class PDFex(object):
    def __init__(self, pdfurl, percent = 0.5):
        self.pdfurl = pdfurl
        ex = extractor.StockFinanceExtractor(pdfurl, percent)
        self.nodes = ex.get_pdf_objects()
        self.headlines = self.getHeadlines(self.nodes, 5)
        self.tablenodes = self.get_table_nodes()
        
    def getPara(self, cond = None, filter_headline = True):
        targethl = []
        res = []
        bol = True
        for hl in self.headlines:
            if not cond is None:
                for c in cond:
                    if c in hl.obj.get_text():
                        targethl.append(hl)
    #     print 'targethl :' + str(len(targethl))
    #     for t in targethl:
    #         print t.obj.get_text()
        if len(targethl) == 0:
            paraList = []
            bol = False
            for node in self.nodes:
                if filter_headline is True:
                    if isinstance(node.obj, pdfextractor.paragraph.Paragraph):
                        paraList.append(node.obj.get_text())
                else:
                    if isinstance(node.obj, pdfextractor.paragraph.Paragraph) or isinstance(node.obj, pdfextractor.paragraph.Headline):
                        paraList.append(node.obj.get_text())
            return bol, paraList

        for t in targethl:
            res.extend(self.get_para_recu(t, filter_headline))
        return bol, res
    
    def get_para_recu(self, node, filter_headline = True):
        res = []
        if isinstance(node.obj, pdfextractor.paragraph.Headline) and filter_headline is False:
            res.append(node.obj.get_text())
        if isinstance(node.obj, pdfextractor.paragraph.Paragraph):
            res.append(node.obj.get_text())
            return res
        child_headlines = self.get_child_headlines(node)
        child_paras = self.get_child_paragraphs(node)
        for c in child_paras:
            res.append(self.filter_text(c.obj.get_text()))
        for c in child_headlines:
            res.extend(self.get_para_recu(c, filter_headline))
        return res

    def filter_text(self, text):
        """
        过滤掉段落长度小于25以及包含非法字符的句子
        """
        if len(text.decode('utf-8')) <= 25:
            return ''
        if u'□' in text or u'√' in text:
            return ''
        return text
    
    def filter_number(self, text):
        """
        去除文字前的数字标号
        """
        regs = [
        u'^第([一|二|三|四|五|六|七|八|九|十]{1,2})节',
        u'^[（|(]{0,1}([一|二|三|四|五|六|七|八|九|十]{1,2})[）|)]',  # 带括号的标题，如（一）、一）仅仅支持到二十
        u'^[（|(]{0,1}([0-9]{1,2})[）|)]',  # 带括号的数字标题，如（1）、1）
        u'^([一|二|三|四|五|六|七|八|九|十]{1,2})、',  # 带顿号的标题，如一、
        u'^([0-9]){1，2}、',  # 带顿号的数字标题，如1、
        ]
        for r in regs:
            text = text.strip(r)
        return text
    
    def is_headline(self, text):
        """
        判断text是否为headline
        """
        regs = [
            u'^第([一|二|三|四|五|六|七|八|九|十]{1,2})节',
            u'^[（|(]{0,1}([一|二|三|四|五|六|七|八|九|十]{1,2})[）|)]',  # 带括号的标题，如（一）、一）仅仅支持到二十
            u'^[（|(]{0,1}([0-9]{1,2})[）|)]',  # 带括号的数字标题，如（1）、1）
            u'^([一|二|三|四|五|六|七|八|九|十]{1,2})、',  # 带顿号的标题，如一、
            u'^([0-9]{1,2})、',  # 带顿号的数字标题，如1、
            ]
        content = text
        for r in regs:
            if re.search(r, u''+content):
                return True
        return False
    
    def get_child_headlines(self, pdf_node):
        '''Get headlines of children of a node
        '''
        headlines = []
        for node in pdf_node.children:
            if isinstance(node.obj, pdfextractor.paragraph.Headline):
                headlines.append(node)
        return headlines

    def get_child_paragraphs(self, pdf_node):
        '''Get paragraphs(node) of children of a node
        '''
        paragraphs = []
        for node in pdf_node.children:
            if isinstance(node.obj, pdfextractor.paragraph.Paragraph):
                paragraphs.append(node)
        return paragraphs
    def get_first_child_paragraph(self, pdf_node):
        """
        get the first paragraph(text) under the given headline
        """
        for node in pdf_node.children:
            if isinstance(node.obj, pdfextractor.paragraph.Paragraph):
                return node.obj.get_text()

    def getHeadlines(self, nodes, level):
        """
        return headline-nodes from given nodes
        """
        re = []
        for node in nodes:
            if isinstance(node.obj, pdfextractor.paragraph.Headline) and node.obj.level <= level:
                re.append(node)
        return re
    
    def get_node_texts(self, pdf_nodes):
        '''
        Given a list of nodes, get the texts of these nodes
        '''
        texts = []
        for node in pdf_nodes:
            text = node.obj.get_text()
            if len(text) <= 25:
                continue
            if u'□' in text or u'√' in text:
                continue
            texts.append(text)
        return texts
    
    def get_paragraph_headline(pdf_node, dis = 1):
        '''Get headline of a paragraph
        Input:
            - pdf_node: a pdf node
        Output:
            - the headline text
        '''
        node = pdf_node
        while dis > 0:
            if isinstance(node.obj, pdfextractor.pdftree.VirtualNode):
                raise BaseException("This node doesn't have headline / distance too big")
            node = node.parent
            dis -= 1
        return node.get_text()
    
    
    def get_table_nodes(self):
        tables = []
        for node in self.nodes:
            if isinstance(node.obj, pdfextractor.table.Table):
                tables.append(node)
        return tables
    
    def get_headline_nodes(self):
        headlines = []
        for node in self.nodes:
            if isinstance(node.obj, pdfextractor.paragraph.Headline):
                headlines.append(node)
        return headlines

    #####################################################################################
    #about table
    #####################################################################################
    def get_table_column_name(self,node):
        """
        return columns name of the given table node
        """
        if not isinstance(node.obj, pdfextractor.table.Table):
            raise TypeError("node is not a table object")
        return node.obj.get_text().split('\n')[0].split('|')
    
    def get_table(self, row_cond, col_cond, headline):
        """
        return the table content match the given condition
        row_cond： 行需要包括的信息
        col_cond： 列需要包括什么信息
        headline： 这个表需要在什么headline下
        """
        for node in self.tablenodes:
            flag = False
            if isinstance(node.parent.obj, pdfextractor.pdftree.VirtualNode):
                continue
            for h in headline:
                if h in node.parent.obj.get_text():
                    flag = True
            if flag is False:
                continue
                
            rows = node.obj.get_text().split('\n')
            index = '.'.join([row[0] for row in rows])
            for r in row_cond:
                if r in index:
                    flag = False
            if flag is True:
                continue
            cols = '.'.join(rows[0])
            for c in col_cond:
                if c in cols:
                    flag = True
            if flag is False:
                continue
            return node.obj.get_text()
        return None
            
    
    def get_table_info(self,node, row_cond = None, col_cond = None, headline = None):
        """
        return the table content match the given condition
        row_cond： 行需要包括的信息
        col_cond： 列需要包括什么信息
        headline： 这个表需要在什么headline下
        """
        content = []
        flag = False
        if isinstance(node.parent.obj, pdfextractor.pdftree.VirtualNode):
            return content
        if not headline is None:
            for h in headline:
                if h in node.parent.obj.get_text():
                    flag = True
        else:
            flag = True
        if flag is False:
            return content
            
        if not isinstance(node.obj, pdfextractor.table.Table):
            raise TypeError("node is not a table object")
        if row_cond is None:
            return content
        if not (type(row_cond) == list):
            raise TypeError("parameter 'match' should be list type")
        if (not col_cond is None) and (not col_cond in node.obj.get_text().split('\n')[0]):
            return content
        content.append(self.get_table_column_name(node))
        rows = node.obj.get_text().split('\n')[1:]
        for r in rows:
            for m in row_cond:
                if m in r:
                    content.append(r.split('|'))
                    
        if len(content) == 1:
            return []
        return content
    
    

In [ ]:
p = PDFex('http://www.neeq.com.cn/disclosure/2017/2017-04-19/1492600473_047614.pdf')


In [428]:
for i in p.getPara(['商业模式'])[1]:
    print i
    print '-'

随着《国家中长期教育改革和发展规划纲要 2010-2020 年》和《国务院关于当前发展学前教育的若干
意见》的出台和落实，我国各地区不断推进学前教育三年行动计划，全国学前教育进入快速普及的发展时
期。2015 年李克强总理提出互联网+行动计划，互联网+教育成为国家大事。五中全会的“十三五”规划
建议提出：“实施国家大数据战略，推进数据资源开放共享。”大数据已明确成为国家战略，教育行业大数
据化和互联网化的方向更加清晰。2016 年全年全国出生人口 1786 万人,比上年多增 131 万人，新生儿的
增速提速和巨大的人口基数为儿童教育产业的强势崛起提供了广阔的生源和需求端基础。
-
随着儿童教育理念逐步走向成熟，以 80/90 后为主的家长群体更加重视儿童的素质教育，注重孩子的
个性化成长；另外，移动互联网让在线教育成为现实，大数据又使得儿童教育的过程和效果都变得更加可
视化，科技幼教开始渗透到儿童教育过程中，家长群体作为互联网用户，在自己孩子在个性化成长和教育
方面的支出具有更加强烈的意愿。
-
本公司所属行业为幼教行业，首倡基于儿童成长教育大数据为基础的个性化教育理念，是行业领先的
儿童个性化教育整体解决方案服务商，是集教育内容研发、互联网技术研发、线上和线下营销、运营服务
支撑于一体的创新型高新技术教育企业。公司致力于帮助早教中心、幼儿园和培训机构提供系统的个性化
教育解决方案、专业的培训服务和精准的运营支持，通过全国各地市的销售渠道，从 B 端到 C 端，进一
步为儿童和家庭提供个性化课程和服务。
-
公司主营业务为面向全国的早教中心、幼儿园和培训机构，提供以儿童成长大数据为基础的个性化教
育解决方案，涵盖产品研发、推广销售、培训服务、后期支撑、数据运营等教育业务全流程。具体包括轻、
中、重三种模式的解决方案：轻量级的个性化解决方案包括面向 B 端以 SaaS 模式（Software as a Service）
提供服务的儿童成长教育相关的各种评量；中量级的个性化方案包括综合评量、个性化方案以及相关教育
专家的在线课堂和咨询，同时支持园所或培训机构构建爱宝儿童个性化成长评量馆；重量级的个性化方案，
是在中量级方案的基础上，既进行了产品种类的拓展，如从儿童综合评量衍生到儿童专注力、体智能、家
庭教育测评等多个方面，又向 B 端提供深度的线上、线下课程，还向 C 

In [74]:
class PDF2Article(PDFex):
    
    def __init__(self, pdfurl, plate = 'hushen', style = 'season'):
        """
        style: ['season', 'year']
        plate: ['hushen', 'xinsanban']
        """
        if style == 'season':
            super(PDF2Article, self).__init__(pdfurl, 0.5)
        else:
            super(PDF2Article, self).__init__(pdfurl, 0.3)
        self.plate = plate
        self.labels = ['导语', '业绩变动原因', '主营业务$业务进展', '未来计划', 'other']
        self.labelmap = {
                         '导语': ['导语','意图_企业基本信息介绍_企业基本信息介绍', '意图_企业活动_公司公告信息', '意图_企业业绩_现状偏好', '意图_企业业绩_现状偏坏'],
                         '业绩变动原因': ['意图_企业业绩_业绩变动原因'],
                         '主营业务$业务进展': ['意图_企业业务_业务介绍', '意图_企业业务_业务规模', '意图_企业业务_业务规划'],
                         '未来计划': ['意图_企业业务_业务前景', '意图_企业业绩_前景偏好','意图_企业业绩_前景偏坏'],
                         'other': ['其他', '意图_评论_风险提示','意图_评论_投资建议', '意图_评论_盈利预测', '意图_企业活动_人事活动', '意图_企业活动_融资活动','意图_企业活动_投资活动']
                        }
        self.labelmapR = {}
        for key, value in self.labelmap.items():
            for v in value:
                self.labelmapR[v] = key
        self.pdfurl = pdfurl
        self.styles = ['season', 'annual']
        self.style = style
        self.companyname = '【公司】'
        if self.plate == 'hushen':
            for node in self.tablenodes:
                content = self.get_table_info(node, row_cond = ['公司的中文简称'], headline=['公司信息'])
                if len(content) == 0:
                    continue
                else:
                    self.companyname = content[1][1]
                    break
        else:
            for node in self.tablenodes:
                content = self.get_table_info(node, row_cond = ['证券简称'], headline = ['公司信息', '基本信息'])
                if len(content) == 0:
                    continue
                else:
                    self.companyname = content[1][1]
                    break
        
        #营业收入：1， 增减：3， 净利润：5，增减：7  总资产：10
        self.ll = ['芥末堆xx月xx日讯，近日，' +  self.companyname + '发布【年|季度】报告，报告期内，' + self.companyname + '营业收入为', 'xx', '元，同比',
                   'xx', '%，归属于上市公司股东的净利润为', 'xx', '元，同比', 'xx', '%。',
                   '截至报告期末，' + self.companyname + '总资产', 'xx', '元。']
        self.reason = []
        self.mainB = []
        self.BProgress = []
        self.future = []
        self.financeTable = []
        self.intents = [self.ll, self.reason, self.mainB, self.BProgress, self.future]
    
    def splitByPeriod(self, P):
        """
        split paragraph to sentences
        """
        senList = sum([t.encode('utf-8').split('。') for t in P],[])
        senList = [t.replace('\n', '') for t in senList]
        senList = filter(lambda x: x != '', senList)
        return senList
    
    
    ########################################################
    #predict intent
    ########################################################
    def getProbility(self, text, itt = '主营业务$业务进展'):
        content = {'content': text}
        intent = requests.post("https://surreal.aidigger.com/api/v1/du/finance",json = content).json()
        intent = intent[0]['features']['intents']
        prob = 0
        for i in intent:
            if i['name'] in self.labelmap[itt]:
                prob += i['prob']
        return prob
        
    
    def getPredict(self, paras, itt = '主营业务$业务进展', num = 2, threshold = 0.5):
        """
        give paragraphs, return paras with given intent that have the most prob 
        """
        text = []
        for p in paras:
            text.append({'content': p})
        intent = requests.post("https://surreal.aidigger.com/api/v1/du/finance",json=text).json()
        intent = {x['content']: x['features']['intents'] for x in intent}
        for key, value in intent.items():
            intent[key] = {x['name']: x['prob'] for x in value}
            intent[key] = self.intentMap(intent[key])
        intentL = {}
        for l in self.labels:
            intentL[l] = {}
        for k, v in intent.items():
            for i, p in v.items():
                intentL[i][k] = p
        for i in intentL.keys():
            intentL[i] = sorted(intentL[i].items(), key=operator.itemgetter(1))
        return [p[0] for p in intentL[itt][-num:] if (p[1] > threshold)]
            
    def intentMap(self, predict):
        """
        将现有意图映射到芥末堆所给的意图上去, 并将权重求和
        """
        result = {}
        for l in self.labels:
            result[l] = 0
        for key in predict.keys():
            result[self.labelmapR[key.encode('utf-8')]] += predict[key]
        return result       
    
    
        
    ########################################################
    #generate article
    ########################################################
    def genLL(self):
        data = []
        for node in self.tablenodes:
            content = self.get_table_info(node, row_cond = ['收入（元）', '营业收入', '归属于上市公司股东的净利润', '总资产', '资产总', '归属于挂牌公司股东的净利润'], headline = ['会计数据','财务指标', '盈利能力'])
            if len(content) == 0:
                continue
            else:
                data = content
                break
        if len(data) == 0:
            self.intents[0] = [''.join(self.ll)]
            return
        for d in data[1:]:
            while ' ' in d:
                d.remove(' ')
            if ('营业收入' in d[0]) or ('收入（元）' in d[0]) :
                num = float(d[1].strip('元').replace(',', ''))
                income = ''
                if num >= 100000000:
                    income = str(round(num / 100000000, 2)) + '亿'
                else:
                    income = str(round(num / 10000, 2)) + '万'
                self.ll[1] = income
                if  '-' in d[3]:
                    self.ll[3] = '减少' + d[3].strip('%- ')
                else:
                    self.ll[3] = '增长' + d[3].strip('% ')
            elif ('归属于上市公司股东的净利润' in d[0]) or ('归属于挂牌公司股东的净利润' == d[0].strip()):
                num = float(d[1].strip('元').replace(',', ''))
                profit = ''
                if num >= 100000000:
                    profit = str(round(num / 100000000, 2)) + '亿'
                else:
                    profit = str(round(num / 10000, 2)) + '万'
                self.ll[5] = profit
                if  '-' in d[3]:
                    self.ll[7] = '减少' + d[3].strip('%- ')
                else:
                    self.ll[7] = '增长' + d[3].strip('% ')
            elif ('总资产' in d[0]) or ('资产总' in d[0]):
                self.ll[10] = d[1]
        if self.plate == 'hushen':
            self.intents[0] = [''.join(self.ll)]
        else:
            self.intents[0] = [''.join(self.ll).split('。')[0] + '。']
                
    def genReason(self):
        if self.style == 'season':
            #问题：pdf将带序号的都认为是headline，并且只取第一行
            reasonTable = []
            reasonT = []
            for table in self.tablenodes:
                r = self.get_table_info(table, ['营业收入', '净利润'], '变动原因')
                if len(r) == 0:
                    continue
                else:
                    reasonTable = r
                    break
            if len(reasonTable) != 0:
                reasonT.append('报告称，')
                for r in reasonTable[1:]:
                    if '营业收入' in r[0]:
                        reasonT.append('营业收入发生变动是由于')
                        reasonT.append(r[4])
                        reasonT.append('。')
                    if '净利润' in r[0]:
                        reasonT.append('净利润发生变动是由于')
                        reasonT.append(r[4])
                        reasonT.append('。')
            rt = ''.join(reasonT)
            b, reasonPara = self.getPara(['变动的情况及原因'], filter_headline = False)
            if b is True:
                removelist = []
                for r in reasonPara:
                    if not ('营业收入' in r or '净利润' in r or '主营业务收入' in r):
                        removelist.append(r)
                for r in removelist:
                    reasonPara.remove(r)

                reasonPara = [self.filter_number(r) for r in reasonPara]
                rp = '\n'.join(reasonPara)
            else:
                rp = ''
            
            self.intents[1].append(rt)
            self.intents[1].append(rp)
        
        else:
            if self.plate == 'hushen':
                b, paras = self.getPara(['讨论与分析'], filter_headline = False)
                if b is False:
                    return 
                paras = self.getPredict(paras, '业绩变动原因', num = 1, threshold = 0.7)
                self.reason.extend(paras)
            else:
                b, paras = self.getPara(['经营情况'], filter_headline = False)
                if b is True:
                    for p in paras:
                        if ('营业收入' in p or '净利润' in p) and ('原因' in p or '因为' in p or '因此' in p):
                            self.intents[1].append(p)
                    return 
                b, paras = self.getPara(['项目重大变动原因'], filter_headline = False)
                if b is True:
                    i = 0
                    while i < len(paras):
                        if '营业收入' in paras[i] or '净利润' in paras[i]:
                            self.intents[1].append(paras[i])
                            if paras[i].endswith('：'):
                                while i < len(paras) - 1 and (not paras[i+1][0].isdigit()):
                                    self.intents[1].append(paras[i+1])
                                    i = i + 1
                        i = i + 1
                    return      
                            

    def genmainB(self):
        if self.style == 'season':
            para = ''
            p = self.getPara(['业务回顾和展望'], filter_headline = True)
            if p[0] is False:
                return
            for s in p[1]:
                if self.filter_text(s) != '':
                    para = s
                    break
            self.mainB.append(para)
            return para
        else:
            if self.plate == 'hushen':
                b, para = self.getPara(['公司业务概要'], filter_headline = True)
                if b is True:
                    mb = ''
                    for p in para:
                        if (self.filter_text(p) != '') and (not p.startswith('公司是否需要')):
                            mb = p
                            break
                    self.mainB.append(mb)
            else:
                b,paras = self.getPara(['商业模式'], filter_headline = False)
                if b is True:
                    removelist = []
                    for p in paras:
                        if type(p) is list:
                            raise BaseException("here")
                        if (not '\n' in p) and ('未发生' in p):
                            removelist.append(p)
                            continue
                        if self.is_headline(p):
                            continue
                        if self.getProbility(p) < 0.3:
                            removelist.append(p)
                    for r in removelist:
                        paras.remove(r)
                    removelist = []
                    for i in range(len(paras)):
                        if self.is_headline(paras[i]) and (i == len(paras) - 1 or self.is_headline(paras[i + 1])):
                            removelist.append(paras[i])
                    for r in removelist:
                        paras.remove(r)
                    self.mainB.extend(paras)
                        
                    
    def genBP(self):
        if self.plate == 'hushen':
#             b1, bp1 = self.getPara(['核心竞争力分析'], filter_headline = True)
#             if b1 is True:
#                 bp1 = [self.filter_text(x) for x in bp1 if self.filter_text(x) != '']
#                 if len(bp1) >= 2:
#                     bp1 = self.getPredict(bp1)
                
#             else:
#                 bp1 = []

            b2, bp2 = self.getPara(['概述'], filter_headline = True)
            if b2 is True:
                bp2 = [self.filter_text(x) for x in bp2 if self.filter_text(x) != '']
#                 bp2 = self.getPredict(bp2)
                if len(bp2) >= 3:
                    bp2 = bp2[:3]
            else:
                bp2 = []
                
            b3, bp3 = self.getPara(['一、经营情况的讨论与分析'], filter_headline = True)
            if b3 is True:
                bp3 = [self.filter_text(x) for x in bp3 if self.filter_text(x) != '']
#                 bp3 = self.getPredict(bp3)
                if len(bp3) >= 3:
                    bp3 = bp3[:3]
            else:
                bp3 = []

#             self.BProgress.extend(bp1)
            self.BProgress.extend(bp2)
            self.BProgress.extend(bp3)
        else:
            b, paras = self.getPara(['经营情况'], filter_headline = False)
            if b is True:
                removelist = []
                for p in paras:
                    if self.is_headline(p):
                        continue
                    if '原因' in p or '因为' in p or '因此' in p or self.getProbility(text = p, itt = '导语') > 0.6:
                        removelist.append(p) 
                for r in removelist:
                    paras.remove(r)
                removelist = []
                for i in range(len(paras)):
                    if self.is_headline(paras[i]) and (i == len(paras) - 1 or self.is_headline(paras[i + 1])):
                        removelist.append(paras[i])
                for r in removelist:
                    paras.remove(r)
                self.BProgress.extend(paras)
            b, paras = self.getPara(['总体回顾'], filter_headline = False)
            if b is True:
                removelist = []
                for p in paras:
                    if self.is_headline:
                        continue
                    if self.getProbility(p) < 0.3:
                        removelist.append(p)
                removelist = []
                for i in range(len(paras)):
                    if self.is_headline(paras[i]) and (i == len(paras) - 1 or self.is_headline(paras[i + 1])):
                        removelist.append(paras[i])
                for r in removelist:
                    paras.remove(r)
                self.BProgress.extend(paras)
        
    def genFuture(self):
        if self.style == 'season':
            para = ''
            p = self.getPara(['业绩的预计'], filter_headline = False)
            if p[0] is False:
                return
            for s in p[1]:
                if self.filter_text(s) != '':
                    para = s
                    break
            self.future.append(para)
        else:
            b, paras = self.getPara(['经营计划', '发展计划', '经营工作计划', '经营管理计划', '发展战略'], filter_headline = False)
            if b is True:
                self.future.extend(paras)

    def genTable(self):
        table = self.get_table(row_cond = [''], col_cond = [''], headline = ['会计数据', '财务指标'])
        if not table is None:
            self.financeTable.append(table)
        
    
    def genArticle(self, style = 'string'):
        """
        获取生成文章。
        style决定是返回一个文章字符串，还是由句子组成的list.(style = 'list')
        """
#         self.genLL()
        self.genReason()
        self.genmainB()
        self.genBP()
        self.genFuture()
        self.genTable()
        article = []
        label = ['导语', '业绩变动原因', '主营业务', '业务进展', '未来计划']
        for i in range(len(self.intents))[1:]:
            article += self.intents[i]
            if style == 'string':
                article.append('\n')
        
        senList = article
        removeList = []
        for i in range(len(senList)):
            senList[i].replace('公司是否需要遵守特殊行业的披露要求' + '\n' + '否', '')
            senList[i].replace('公司是否需要遵守特殊行业的披露要求否', '')
            senList[i].replace('公司是否需要遵守特殊行业的披露要求是', '')
            if '详见' in senList[i]:
                removeList.append(senList[i])
        for r in removeList:
            senList.remove(r)
        if style == 'string':
            return '\n'.join(senList)
        else :
            return senList

    
    def getAllMB(self):
        """
        获取pdf中-业务概要-中的所有内容
        """
        return self.getPara(['公司业务概要'], filter_headline = False)
    
    def getALLBP(self):
        """
        获取pdf中-核心竞争力分析-中的所有内容
        """
        return self.getPara(['核心竞争力分析'], filter_headline = False)
    
    
    

In [73]:
for node in pdf.tablenodes:
    content = pdf.get_table_info(node, ['归属于上市公司股东的净利润'], headline=['会计数据'])
    if len(content) == 0:
        continue
    else:
        data = content
        break
for d in data[1]:
    print d

归属于上市公司股东的净利润（元
） 182,233,544.71
 119,471,450.39
 52.53
 105,190,067.9


In [67]:
# for i in pdf2jmd['http://disclosure.szse.cn/finalpage/2017-04-25/1203379575.PDF']['content']:
#     print i
#     print '-'
# print '**************'
pdf = PDF2Article('http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF', plate = 'hushen', style='year')
# for node in pdf.nodes[1:]:
#     if isinstance(node.obj, pdfextractor.paragraph.Headline) and '总体回顾' in node.obj.get_text():
#         target = node
#         break
        
# print target.parent
# for i in target.children:
#     print i
# print pdf.genArticle()

Download pdf time: 7s
download http status code: 200
Pdfminer Parsed time: 19s
Construct Pdf Tree time: 0s


In [653]:
data = []
for node in pdf.tablenodes:
    content = pdf.get_table_info(node, row_cond = ['收入（元）', '营业收入', '归属于上市公司股东的净利润', '总资产', '资产总', '归属于挂牌公司股东的净利润'], headline = ['会计数据','财务指标', '盈利能力'])
    if len(content) == 0:
        continue
    else:
        data = content
        break
        
for i in data:
    for d in i:
        print d == ' '
    print '******'

False
False
False
False
******
False
True
False
False
False
******
False
False
True
False
False
******
False
True
False
False
False
******


In [680]:
a = [' ', ' ', 'ds']
a.remove(' ')
a

[' ', 'ds']

In [66]:
yearArticle = ['http://www.cninfo.com.cn/finalpage/2017-04-24/1203360123.PDF', 'http://www.cninfo.com.cn/finalpage/2017-03-28/1203210341.PDF',
              'http://www.cninfo.com.cn/finalpage/2017-03-30/1203223635.PDF', 'http://www.cninfo.com.cn/finalpage/2017-03-21/1203180898.PDF',
              'http://www.cninfo.com.cn/finalpage/2017-03-21/1203181705.PDF', 'http://www.cninfo.com.cn/finalpage/2017-04-11/1203276830.PDF',
              'http://www.cninfo.com.cn/finalpage/2017-04-29/1203428096.PDF', 'http://www.cninfo.com.cn/finalpage/2017-03-07/1203134952.PDF',
              'http://www.cninfo.com.cn/finalpage/2017-04-18/1203314378.PDF']
i = 0
for u in yearArticle:
    fileobj = open('/home/zhuyuhe/mydata/jmd/percent/articleGen/year/yearArticle' + str(i), 'wb')
    fileobj.write(u)
    fileobj.write('\n生成文章如下: ')
    fileobj.write('-----------------\n')
    try:
        article = PDF2Article(u, 'year').genArticle()
    except BaseException:
        print '*********PDF parse error*************'
        fileobj.close()
        continue
    fileobj.write(article)
    fileobj.close()
    i += 1
    print '*****************************************************************'

Download pdf time: 12s
download http status code: 200
Pdfminer Parsed time: 19s
Construct Pdf Tree time: 0s
*****************************************************************
Download pdf time: 0s
download http status code: 200
Pdfminer Parsed time: 16s
Construct Pdf Tree time: 0s
*****************************************************************
Download pdf time: 0s
download http status code: 200
Pdfminer Parsed time: 9s
Construct Pdf Tree time: 0s
*****************************************************************
Download pdf time: 4s
download http status code: 200
Pdfminer Parsed time: 12s
*********PDF parse error*************
Download pdf time: 2s
download http status code: 200
Pdfminer Parsed time: 20s
Construct Pdf Tree time: 0s
*****************************************************************
Download pdf time: 2s
download http status code: 200
Pdfminer Parsed time: 8s
Construct Pdf Tree time: 0s
*********PDF parse error*************
Download pdf time: 3s
download http status code

In [30]:
urls = ['http://www.cninfo.com.cn/finalpage/2017-04-26/1203387778.PDF', 'http://www.cninfo.com.cn/finalpage/2017-10-31/1204097927.PDF', 
        'http://www.cninfo.com.cn/finalpage/2017-04-25/1203375291.PDF', 'http://www.cninfo.com.cn/finalpage/2017-04-28/1203411234.PDF',
       'http://www.cninfo.com.cn/finalpage/2017-04-29/1203425130.PDF', 'http://www.cninfo.com.cn/finalpage/2017-04-29/1203423109.PDF',
       'http://www.cninfo.com.cn/finalpage/2017-10-28/1204085108.PDF', 'http://disclosure.szse.cn/finalpage/2016-10-25/1202780794.PDF',
        'http://disclosure.szse.cn/finalpage/2016-04-26/1202239317.PDF', 'http://disclosure.szse.cn/finalpage/2016-10-27/1202793355.PDF',
        'http://disclosure.szse.cn/finalpage/2017-04-26/1203386046.PDF']
fileobj = open('/home/zhuyuhe/mydata/jmd/percent/articleGen/season/seasonArticle', 'wb')
for u in urls:
    fileobj.write(u)
    fileobj.write('\n生成文章如下: ')
    fileobj.write('-----------------\n')
    fileobj.write(PDF2Article(u).genArticle())
    fileobj.write('\n\n\n')
fileobj.close()

Download pdf time: 1s
download http status code: 200
Pdfminer Parsed time: 7s
Construct Pdf Tree time: 0s
Download pdf time: 3s
download http status code: 200
Pdfminer Parsed time: 1s
Construct Pdf Tree time: 0s
Download pdf time: 3s
download http status code: 200
Pdfminer Parsed time: 4s
Construct Pdf Tree time: 0s
Download pdf time: 5s
download http status code: 200
Pdfminer Parsed time: 5s
Construct Pdf Tree time: 0s
Download pdf time: 0s
download http status code: 200
Pdfminer Parsed time: 3s
Construct Pdf Tree time: 0s
Download pdf time: 0s
download http status code: 200
Pdfminer Parsed time: 5s
Construct Pdf Tree time: 0s
Download pdf time: 0s
download http status code: 200
Pdfminer Parsed time: 6s
Construct Pdf Tree time: 0s
Download pdf time: 3s
download http status code: 200
Pdfminer Parsed time: 4s
Construct Pdf Tree time: 0s
Download pdf time: 2s
download http status code: 200
Pdfminer Parsed time: 6s
Construct Pdf Tree time: 0s
Download pdf time: 2s
download http status cod

In [ ]:
for i in p.getPara(['教育服务板块'])[1]:
    print i
    print '-'

In [406]:
p = PDF2Article('http://disclosure.szse.cn/finalpage/2017-04-11/1203274908.PDF', 'yera')
print p.genArticle()

Download pdf time: 2s
download http status code: 200
Pdfminer Parsed time: 12s
Construct Pdf Tree time: 0s
芥末堆xx月xx日讯，近日，【公司】发布【年|季度】报告，其营业收入为389,177,122.03 元，同比增长5.45% %，归属于上市公司股东的净利润为33,284,823.81 元，同比增长7.39% %。截至报告期末，【公司】总资产928,997,567.95 元。


 |2016年 |2015年 |本年比上年增减 |2014年 
营业收入（元） |389,177,122.03 |369,061,636.55 |5.45% |353,082,445.53 
归属于上市公司股东的净利润（元） |33,284,823.81 |30,993,823.55 |7.39% |25,686,859.50 
归属于上市公司股东的扣除非经常性损益的净利润（元） |21,670,738.54 |25,525,648.58 |-15.10% |22,978,926.76 
经营活动产生的现金流量净额（元） |42,017,806.99 |46,148,262.48 |-8.95% |5,160,201.73 
基本每股收益（元/股） |0.1325 |0.1285 |3.11% |0.1065 
稀释每股收益（元/股） |0.1325 |0.1285 |3.11% |0.1065 
加权平均净资产收益率 |4.23% |5.60% |-1.37% |4.85% 
 |2016年末 |2015年末 |本年末比上年末 增减 |2014年末 
资产总额（元） |928,997,567.95 |889,518,477.17 |4.44% |668,431,599.38 
归属于上市公司股东的净资产（元） |800,245,430.48 |772,991,563.79 |3.53% |540,021,669.30 




报告期内，公司主要从事钢琴的研发、制造、销售与服务，同时加大智能钢琴的研究与开发，拓展艺术教育培训产业等。从公司创立至今，公司完成了四大转变：一是产品结构方面，完成了从钢琴零配件到钢琴核心部件、整琴制造、精品钢琴的转变，目前正在研制智能钢琴；二是品牌建设方

In [407]:
for info in year_A:
    if info['pdf_url'] == 'http://disclosure.szse.cn/finalpage/2017-04-11/1203274908.PDF':
        print info['content']


芥末堆4月11日讯，昨日海伦钢琴（SZ:300329）发布2016年度报告，数据显示，2016年海伦钢琴实现营业收入3.89亿元，净利润为3328.48万元。 
海伦钢琴从事钢琴研发、制造、销售与服务。2014年起，公司积极拓展艺术教育培训项目，主要通过与各地培训机构签署合作协议的方式开展运营。艺术教育培训课程体系定位为艺术启蒙、音乐教育、美术教育与舞蹈教育。 
根据财报，2016年，海伦钢琴营收达3.89亿元，同比增长5.45%，归属于上市公司股东的净利润为3328.48万元，同比增长7.39%。从营收构成上看，只能看出各类乐器的销售营收占比，无法看出艺术教育培训的营收情况。 
财报分析增长原因认为，首先是政策因素。国家积极推动文化产业发展，钢琴产业升级、与互联网的结合等得到了国家政策的有力扶持。报告期内，海伦钢琴获得了 “引进外国专家项目资助”、“文化产业项目获国家市级奖励资金补助”、“中央文化产业发展专项资金政府补助”、“企业技术创新团队专项资助”、“引智项目配套扶持资金”等多项政策性项目奖励。 
技术优势也是增长的一个原因。财报称，海伦钢琴在钢琴设计和制造技术方面吸收了欧洲钢琴的先进设计理念，逐步形成了将现代高新技术与钢琴传统个性化加工工艺相结合的技术特点，建立了一整套独立的创新技术体系。报告期内，共设立了5项新产品项目，新申请6项专利，授权取得6项专利。 
在新的业务方向上，海伦钢琴还努力推进智能钢琴及在线教育平台等研发工作。依托北京邮电大学的技术，改造和拓展智能钢琴的产品结构和使用功能，开展在线教育平台研究项目。报告期内，海伦钢琴的“6+1”智能钢琴教室产品也已面市。该产品能够实现多人同时线上签到教学授课。现阶段课程主要针对幼儿初学者，后续还会增设中青年、老年学习者课程。 
艺术教育培训是海伦钢琴未来仍然会重点拓展的产业链。财报介绍，海伦艺术教育投资公司成立五大中心来支持各地艺术教育培训日常运营工作。这五大中心分别为产品研发与教师培养中心、学校经营与管理培训中心、品牌管理与整合传播中心、大型活动与文化传媒中心、整体运营与财务行政中心。 
财报分析，在钢琴进出口的格局逆转下，国产钢琴开始寻求转型之路。借助资本市场，拓展社会音乐教育项目。通过对教育资源和课件研发体系的整合与把握，创造新的市场增量。 
在提供智能钢琴产品的同时，还构建增值教育服务，进一步吸引

In [55]:
pdfurl_filter = []
for y in range(len(data)):
    if data[y]['quarter'] == '2' and data[y]['year'] == '2016':
        pdfurl_filter.append(data[y]['pdf_url'])

In [375]:
overlapPDF = []
for block in overlaplist:
    overlapPDF.append(block[0]['pdf_url'])
overlapPDF

[u'http://disclosure.szse.cn/finalpage/2017-04-24/1203360123.PDF',
 u'http://disclosure.szse.cn/finalpage/2017-04-11/1203274908.PDF',
 u'http://disclosure.szse.cn/finalpage/2017-08-10/1203778984.PDF',
 u'http://disclosure.szse.cn/finalpage/2017-03-21/1203181705.PDF',
 u'http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF',
 u'http://disclosure.szse.cn/finalpage/2016-08-26/1202622898.PDF',
 u'http://static.sse.com.cn/disclosure/listedinfo/announcement/c/2016-08-25/600661_2016_z.pdf']

In [402]:
overlappercent = []
print len(overlaplist)
for i in range(len(overlaplist)):
    block = overlaplist[i]
    jmd = block[0]['content'].strip().encode('utf-8').split('。')
    jmd = [x.replace('\n','') for x in jmd]
    jmt = block[1]['content'].strip().encode('utf-8').split('。')
    jmt = [x.replace('\n','') for x in jmt]
    
    count = 0
    for text in jmd:
        if not (len(text) == 0):
            if isMatch(text, jmt, 0.6):
                count+=1
    print str(count * 1.0/ jmd.__len__() * 100) + '% 的重叠率'
    overlappercent.append(count * 1.0/ jmd.__len__())
    print('***************************')
    
print '平均重叠率为' + str(np.mean(overlappercent))


14
2.32558139535% 的重叠率
***************************
47.619047619% 的重叠率
***************************
0.0% 的重叠率
***************************
16.6666666667% 的重叠率
***************************
23.0769230769% 的重叠率
***************************
29.4117647059% 的重叠率
***************************
46.1538461538% 的重叠率
***************************
73.3333333333% 的重叠率
***************************
38.0952380952% 的重叠率
***************************
9.09090909091% 的重叠率
***************************
27.7777777778% 的重叠率
***************************
35.7142857143% 的重叠率
***************************
16.6666666667% 的重叠率
***************************
88.2352941176% 的重叠率
***************************
平均重叠率为0.324405238867


In [400]:
overlaplist = []
mapdic = {}
for i in range(len(jmt_jmd_mapping)):
    y = jmt_jmd_mapping[i]
    key = y['code'] + '-' + y['year'] + '-' + y['quarter']
    if key in mapdic.keys():
        block = []
        block.append(mapdic[key])
        block.append(y)
        overlaplist.append(block)
    else:
        mapdic[key] = y

In [58]:
fpk = open('/home/zhuyuhe/mydata/jmd/jmt_jmd_mappings2.pkl', 'rb')
jmt_jmd_mapping = pickle.load(fpk)

data = []
for i in range(len(jmt_jmd_mapping)):
    if ((jmt_jmd_mapping[i]['quarter'] == '4' or jmt_jmd_mapping[i]['quarter'] == '2')) and 'jmd_url' in jmt_jmd_mapping[i].keys():
        data.append(jmt_jmd_mapping[i])
        
content_url_map = {}
for d in data:
    content_url_map[d['pdf_url']] = {}
    content_url_map[d['pdf_url']]['content'] = d['content']
    content_url_map[d['pdf_url']]['finance_type'] = d['type']

In [57]:
data[0].keys()

[u'code',
 u'name',
 u'pdf_url',
 u'content',
 u'jmd_url',
 u'year',
 u'quarter',
 u'type']

In [75]:
fpk = open('/home/zhuyuhe/mydata/jmd/jmt_jmd_mappings2.pkl', 'rb')
jmt_jmd_mapping = pickle.load(fpk)


def MatchTwoText(t1, t2, threshold = 0.5):
    """
    检查两个句子是否匹配
    """
    ct1 = ' '.join(jieba.cut(t1)).split()
    ct2 = ' '.join(jieba.cut(t2)).split()
    commonL = set(ct1).intersection(set(ct2)).__len__()
    if commonL * 1.0 / set(ct1).__len__() > threshold:
        return True
    return False
    
def isMatch(text, pdfTextList, threshold = 0.5):
    """
    @parameter text: 待检查句子
    @parameter pdfTextList: PDF句子与表格信息列表
    @parameter threshold: 相似度为多少时返回true
    @return： boolean值，pdf列表中是否有与待检查句子非常相似的句子
    """
    for pdftext in pdfTextList:
        if MatchTwoText(text, pdftext, threshold):
            return True
    return False

data = []
for i in range(len(jmt_jmd_mapping)):
    if ((jmt_jmd_mapping[i]['quarter'] == '4' or jmt_jmd_mapping[i]['quarter'] == '2')) and 'jmd_url' in jmt_jmd_mapping[i].keys():
        data.append(jmt_jmd_mapping[i])
        
content_url_map = {}
for d in data:
    content_url_map[d['pdf_url']] = {}
    content_url_map[d['pdf_url']]['content'] = d['content']
    content_url_map[d['pdf_url']]['finance_type'] = d['type']

urls = content_url_map.keys()

jmt_jmd_data = {}

def parsePDF(urls,style):  
    jmt_jmd_data[style] = {}
    num = 1
    for pdfurl in urls:
        if pdfurl in pdfurl_filter:
            continue
            
        print 'article ' + str(num)
        num +=1
        print pdfurl
        jmt_jmd_data[style][pdfurl] = {}
        
        content = content_url_map[pdfurl]['content']
        plate = content_url_map[pdfurl]['finance_type']
        contentlen = 0
#         if len(content) == 1:
        cxkj = content
        #print cxkj
        cxkj = cxkj.strip().encode('utf-8').split('。')
        cxkj = [x.replace('\n','') for x in cxkj]
        contentlen = len(cxkj)
#         else:
#             jmd = content['content'].strip().encode('utf-8').split('。')
#             jmd = [x.replace('\n','') for x in jmd]
#             jmt = content[1]['content'].strip().encode('utf-8').split('。')
#             jmt = [x.replace('\n','') for x in jmt]
#             cxkj = []
#             count = 0
#             jmt_len = len(jmt)
#             jmd_len = len(jmd)
#             removelist = []
#             for text in jmd:
#                 if not (len(text) == 0):
#                     if isMatch(text, jmt, 0.6):
#                         count+=1
#                         removelist.append(text)
#             for r in removelist:
#                 jmd.remove(r)
#             cxkj += jmd
#             cxkj += jmt
#             contentlen = (jmt_len + jmd_len) / 2
            

        try :
            p = PDF2Article(pdfurl, plate, style)
        except BaseException:
            print 'error extractPDF'
            continue

        print "------"
        pdflist = p.getPara(filter_headline=True)[1]
        jmt_jmd_data[style][pdfurl]['content'] = content_url_map[pdfurl]
        jmt_jmd_data[style][pdfurl]['para'] = pdflist
        jmt_jmd_data[style][pdfurl]['sen'] = p.splitByPeriod(pdflist)
        jmt_jmd_data[style][pdfurl]['recArticle'] = p.genArticle('list')
        jmt_jmd_data[style][pdfurl]['recArticleSen'] = p.splitByPeriod(jmt_jmd_data[style][pdfurl]['recArticle'])
        jmt_jmd_data[style][pdfurl]['mapcontent'] = []
        jmt_jmd_data[style][pdfurl]['mapcontentlen'] = contentlen
        
        count = 0
        for text in cxkj:
            if not (len(text) == 0):
                if isMatch(text, pdflist, 0.6):
                    jmt_jmd_data[style][pdfurl]['mapcontent'].append(text)
                    count+=1
            
        print str(count * 1.0/ cxkj.__len__() * 100) + '% 的句子来自PDF'
        print('***************************')
    
parsePDF(content_url_map.keys(), 'year')

article 1
http://www.cninfo.com.cn/finalpage/2017-04-19/1203340942.PDF
Download pdf time: 1s
download http status code: 200
Pdfminer Parsed time: 6s
Construct Pdf Tree time: 0s
------
8.33333333333% 的句子来自PDF
***************************
article 2
http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF
Download pdf time: 2s
download http status code: 200
Pdfminer Parsed time: 20s
Construct Pdf Tree time: 0s
------
66.6666666667% 的句子来自PDF
***************************
article 3
http://static.sse.com.cn/disclosure/listedinfo/announcement/c/2017-04-15/601801_2016_n.pdf
Download pdf time: 2s
download http status code: 200
Pdfminer Parsed time: 15s
Construct Pdf Tree time: 0s
------
50.0% 的句子来自PDF
***************************
article 4
http://www.cninfo.com.cn/finalpage/2017-04-19/1203341493.PDF
Download pdf time: 4s
download http status code: 200
Pdfminer Parsed time: 13s
Construct Pdf Tree time: 0s
------
18.1818181818% 的句子来自PDF
***************************
article 5
http://www.cninfo.com

In [97]:
key = 'http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF'
for m in jmt_jmd_data['year'][key]['mapcontent']:
    print m
    print '-'
print '**************************************'
print jmt_jmd_data['year'][key]['content']['content']
print '**************************************'
for r in jmt_jmd_data['year'][key]['recArticle']:
    print r
    print '-'

 在幼教行业，威创的目标是打造喜达屋式的多品牌幼教管理集团，逐步完善公司在幼教行业的生态体系布局
-
报告期内，公司形成了线上以家园共育平台贝聊为主的流量平台，线下以幼儿园品牌运营主体红缨教育和金色摇篮为主的运营服务平台
-
依托已有体系，威创打通园所家庭端的入口， 构建儿童成长平台
-
 在幼儿园领域，威创主要的服务模式包括品牌加盟、品牌联盟、托管加盟三种形态，截止报告期末，公司旗下的幼教品牌共拥有加盟儿园接近 4000 家
-
 在资本方面，2016 年 1 月，威创启动非公开发行股份融资，并于 2017 年 1 月获得证监会审核通过，据了解，募集资金将用于儿童艺体培训中心的建设
-
 红缨和金色摇篮完成对赌 年报显示，2015 年度和 2016 年度，红缨教育和金色摇篮实际利润均较大幅度超过并购业绩承诺
-
 >>红缨教育 红缨教育 2016 年营业收入 2.86 亿元，较去年同期增长 48.12%；归属于公司所有者的净利润 1.02 亿元，较去年同期增长 80.76%，截止报告期末共有连锁幼儿园 6 所，红缨品牌加盟园所 1490 所，悠久(Yojo)联盟园所 1695 所, 悠久(Yojo)代理加盟商 166 家
-
其中悠久(Yojo)联盟园从 2015 年的 396 所增加至 2016 年的 1695 所，其商品销售从 2015 年的 133.41 万元发展到 2016 年的 2793.72 万元； 2、加大新产品研发力度，不断推出新品从而提升单园销售收入，2016 年度红缨教育开发悠久(Yojo)联盟园定制的《好未来阅读》、《创意美术》、《思维数学》等系列课程，大幅提高悠久(Yojo)联盟园的商品销售客单价； 3、旗舰园的深入督导带动客单价的大幅提升
-
2016 年，旗舰园的客单价从 11.18 万元增至 17.65万元，主要原因在于，红缨教育对旗舰园在装修设计、教学管理以及教师培训的管理介入更深
-
 >>金色摇篮 截止报告期末，金色摇篮共有托管幼儿园 19 所，加盟幼儿园 410 所，托管小学 2 所，品牌加盟小学 3 所，直营早教中心 2 所，加盟早教中心 10 所，区域城市合伙人 6 家
-
 2016 年度，金色摇篮的商品销售业务收入为 1569.13 万元，以金色摇篮自主研发的课程销售为主，这部分收入为 1406.16 万元
-
服务业

In [84]:
jmt_jmd_data['year'].keys()

[u'http://www.cninfo.com.cn/finalpage/2017-04-19/1203341493.PDF',
 u'http://www.cninfo.com.cn/finalpage/2017-04-21/1203369775.PDF',
 u'http://disclosure.szse.cn/finalpage/2017-03-21/1203181705.PDF',
 u'http://www.cninfo.com.cn/finalpage/2017-05-10/1203508904.PDF',
 u'http://www.cninfo.com.cn/finalpage/2017-06-20/1203635303.PDF',
 u'http://disclosure.szse.cn/finalpage/2017-04-24/1203360123.PDF',
 u'http://disclosure.szse.cn/finalpage/2017-08-10/1203778984.PDF',
 u'http://disclosure.szse.cn/finalpage/2017-03-18/1203174359.PDF',
 u'http://disclosure.szse.cn/finalpage/2017-03-28/1203210341.PDF',
 u'http://www.cninfo.com.cn/finalpage/2017-04-11/1203285646.PDF',
 u'http://www.cninfo.com.cn/finalpage/2017-04-19/1203340942.PDF',
 u'http://disclosure.szse.cn/finalpage/2017-07-28/1203739610.PDF',
 u'http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF',
 u'http://static.sse.com.cn/disclosure/listedinfo/announcement/c/2017-04-15/601801_2016_n.pdf',
 u'http://www.cninfo.com.cn/finalpage/2

In [85]:
count = 0
for j in jmt_jmd_data['year'].keys():
    if 'mapcontent' not in jmt_jmd_data['year'][j].keys():
        continue
    count += 1
print count

19


In [ ]:
# k = jmt_jmd_data['year'].keys()[1]
k = 'http://disclosure.szse.cn/finalpage/2017-07-28/1203739610.PDF'
print k
for i in jmt_jmd_data['year'][k]['content']:
    print i
    print '-'
for i in jmt_jmd_data['year'][k]['mapcontent']:
    print i
    print '-'
print '**************************************************'
for i in jmt_jmd_data['year'][k]['recArticleSen']:
    print i
    print '-'

In [ ]:
print year_A[0]['content']

In [78]:
with open('/home/zhuyuhe/mydata/jmd/percent/jmt_jmd_mappings2_processed_xsb_A_year.pkl', 'wb') as pickle_file:
     pickle.dump(jmt_jmd_data, pickle_file , protocol=2)

In [260]:
p = PDF2Article('http://disclosure.szse.cn/finalpage/2017-03-18/1203174359.PDF', 'year')
print p.genArticle()

Download pdf time: 7s
download http status code: 200
Pdfminer Parsed time: 8s
Construct Pdf Tree time: 0s
芥末堆xx月xx日讯，近日，【公司】发布【年|季度】报告，其营业收入为993,842,901.37元，同比增长 528,273,545.43%，归属于上市公司股东的净利润为34,926,666.56元，同比增长 19,799,591.29%。截至报告期末，【公司】总资产1,795,628,003.46元。


2015年 |本年比上年增减 |2014年 
 |2016年 |调整前 |调整后 |调整后 |调整前 |调整后 
营业收入（元） |993,842,901.37| 528,273,545.43| 528,273,545.43| 88.13| 589,398,426.7| 589,398,426.7
归属于上市公司股东的净利润（元） |34,926,666.56| 19,799,591.29| 19,799,591.29| 76.40| 71,591,324.0| 71,591,324.0
归属于上市公司股东的扣除非经常性损益的净利润（元|33,828,929.4） | 14,098,658.29| 14,098,658.2| 139.94| 37,472,782.5| 37,472,782.5
经营活动产生的现金流量净额（元） |169,139,076.04| 154,479,178.38| 154,479,178.38| 9.49| 103,745,034.6| 103,745,034.6
基本每股收益（元/股） |0.0781| 0.0443| 0.0443| 76.30| 0.16| 0.16
稀释每股收益（元/股） |0.0781| 0.0443| 0.0443| 76.30| 0.16| 0.16
加权平均净资产收益率 |3.96| 2.27%| 2.27| 1.69| 8.40| 8.40
2015年末 |本年末比上年末增减 |2014年末 
 |2016年末 |调整前 |调整后 |调整后 |调整前 |调整后 
总资产（元） |1,795,628,003.46|1,956,925,372. 18|1,956,925,372. 18|-8.2

In [ ]:
for key, value in year_A[8].items():
    print key + ': ' + value

In [103]:
fp = open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_map_headline.pkl', 'rb')
pdf2jmd = pickle.load(fp)['year']

In [125]:
for url in pdf2jmd.keys():
    article = PDF2Article(url, 'year').genArticle()
    pdf2jmd[url]['recpara'] = article.split('\n')

Download pdf time: 5s
download http status code: 200
Pdfminer Parsed time: 11s
Construct Pdf Tree time: 0s
Download pdf time: 1s
download http status code: 200
Pdfminer Parsed time: 5s
Construct Pdf Tree time: 0s


TypeError: sequence item 0: expected string, NoneType found

In [22]:
import requests
text = {'content':'''立思辰教育秉承“以科技和人文改变教育”的历史使命，坚持“做大智慧教育，做强教育服务，做精未来
学校，做实教育生态”的发展路径，利用科技手段打通校内校外、线上线下、国内国外的学习场景，让学生
们高效学习、快乐成长，最终实现“激发成就亿万青少年”的宏伟愿景。

'''}
query = requests.post("https://surreal.aidigger.com/api/v1/du/finance",json=text).json()
# print query[0]
predict = query[0]['features']['intents']
# prob = [x['prob'] for x in predict]
# i = max(prob)
# intent = predict[prob.index(i)]['name']
# print intent
for i in predict:
    print i['name'], i['prob']

意图_企业业绩_现状偏好 0.00548229456175
意图_企业业绩_现状偏坏 0.00516564432715
意图_企业业务_业务前景 0.0471579249175
意图_企业业务_业务规划 0.327638040848
意图_企业业务_业务规模 0.0514242802499
意图_企业业务_业务介绍 0.470186824232
意图_企业活动_公司公告信息 0.00415167844841
意图_企业活动_投资活动 0.0599718276385
意图_企业业绩_前景偏好 0.00204666107317
意图_企业业绩_前景偏坏 0.000461056441688
意图_企业活动_人事活动 0.000933456380522
意图_企业活动_融资活动 0.00214735332778
意图_企业基本信息介绍_企业基本信息介绍 0.00534654128586
意图_企业业绩_业绩变动原因 0.0133105201692
其他 0.000679959637695
意图_评论_风险提示 0.000247481227789
意图_评论_投资建议 0.000662603903443
意图_评论_盈利预测 0.0010989582416
导语 0.00188689308786


## A股季报、年报和半年报 √
## 新三板 ×

In [484]:
content_url_map.keys()

[u'http://static.sse.com.cn/disclosure/listedinfo/announcement/c/2017-08-30/600661_2017_z.pdf',
 u'http://disclosure.szse.cn/finalpage/2017-08-19/1203815519.PDF',
 u'http://static.sse.com.cn/disclosure/listedinfo/announcement/c/2017-08-31/601801_2017_z.pdf',
 u'http://disclosure.szse.cn/finalpage/2017-04-12/1203282865.PDF',
 u'http://disclosure.szse.cn/finalpage/2016-08-24/1202610476.PDF',
 u'http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF',
 u'http://static.sse.com.cn/disclosure/listedinfo/announcement/c/2017-04-15/601801_2016_n.pdf',
 u'http://disclosure.szse.cn/finalpage/2017-04-28/1203414826.PDF',
 u'http://static.sse.com.cn/disclosure/listedinfo/announcement/c/2017-08-10/600551_2017_z.pdf',
 u'http://disclosure.szse.cn/finalpage/2016-08-13/1202563205.PDF',
 u'http://static.sse.com.cn/disclosure/listedinfo/announcement/c/2016-08-25/600661_2016_z.pdf',
 u'http://disclosure.szse.cn/finalpage/2017-08-22/1203848489.PDF',
 u'http://disclosure.szse.cn/finalpage/2016-08-27/1